In [25]:
import torch.nn as nn
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from omegaconf import DictConfig
from loguru import logger

class MirrorUNet(nn.Module):
    def __init__(self):
        super().__init__()
        norm = "instance"
        self.pathology_model = UNet(
            spatial_dims=3,
            in_channels=2,
            out_channels=2,
            channels=(32,64,128,256),
            strides=(2,2,2),
            num_res_units=2,
            norm=norm,
        )
        self.anatomy_model = UNet(
            spatial_dims=3,
            in_channels=1, # ct
            out_channels=145,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            # num_res_units=num_res_units,
            norm="instance",
        )
            
m = MirrorUNet()
m.pathology_model.model
len(m.pathology_model.strides)

3

In [22]:
def get_last_conv(layer, strides):
    """
    Dynamically construct the path to the last convolution layer
    and evaluate it to access the target Conv3d layer.

    Args:
        layer: The input SkipConnection layer.
        strides: The number of strides (depth of traversal).
    Returns:
        The last Conv3d layer.
    """
    #submodule[1] -> submodule[1] -> ...
    path = "layer"
    for _ in range(strides - 1):
        path += ".submodule[1]"
    path += ".submodule.conv.unit0.conv"  
    
    result = eval(path)
    return result
from monai.networks.layers import Norm, SkipConnection
for i, layer in enumerate(m.pathology_model.model):
    # print(f"layer {i}: {layer}")
    if i == 1:
        if isinstance(layer, SkipConnection):
            # last_conv = layer.submodule[1].submodule[1].submodule.conv.unit0.conv
            last_conv = get_last_conv(layer, 3)
            print(last_conv)

Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
